In [5]:
import ee
import geemap
import os

# Authenticate to Earth Engine
try:
  ee.Initialize(project='envr451-2024')
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='envr451-2024')

# Planet
Planet provides us with 4.77m resolution all the way to Jan 2016.
https://developers.google.com/earth-engine/datasets/catalog/projects_planet-nicfi_assets_basemaps_americas

Note - with the NICFI project, we can only get biannual images up until 2020; in 2020 we get
planet_medres_normalized_analytic_2020-06_2020-08_mosaic
planet_medres_normalized_analytic_2020-09_mosaic
planet_medres_normalized_analytic_2020-10_mosaic
planet_medres_normalized_analytic_2020-11_mosaic
planet_medres_normalized_analytic_2020-12_mo

Here, I am considering medians for all years. Consider, however, that some images have clouds, and we expect worse quality for the images from 2015-2020 since 2015-2019 only has 2 images per year, and 2020 has 5, while for 2021-2024 we get all 12 images for the year.

This can be improved once we get all images from McGill. Also, these should be at 3.7m resolution.saic

In [260]:
piura_shp = ee.FeatureCollection('projects/envr451-2024/assets/piura').geometry()
basemap = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas').filter(ee.Filter.date('2015-01-01', '2024-12-31')) \
            .map(lambda image: image.clip(piura_shp))

# Define the years for which you want to calculate geometric medians.
years = ee.List.sequence(2015, 2024)

# Function to calculate geometric median for each year.
def calculate_median(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    yearly_collection = basemap.filterDate(start_date, end_date)
    median = yearly_collection.reduce(ee.Reducer.median())
    return median.set('year', year)

# Map over the list of years and calculate geometric medians.
yearly_medians = ee.ImageCollection(years.map(calculate_median)).map(lambda image: image.clip(piura_shp))
yearly_medians_list = yearly_medians.toList(yearly_medians.size())

# Get the number of images.
num_images = yearly_medians_list.size().getInfo()

# Loop over the list and export each image.
for i in range(num_images):
    year = years.get(i).getInfo()
    task = ee.batch.Export.image.toDrive(
        image=ee.Image(yearly_medians_list.get(i)),
        description = 'Planet_Satellite_5m_median_' + str(year),
        fileNamePrefix = 'Planet_Satellite_5m_median_' + str(year),
        region=piura_shp,
        scale=5,
        maxPixels=1e13
    )
    task.start()

# vis_basemap = {'bands': ['R_median', 'G_median', 'B_median'], 'min': 64, 'max': 5454, 'gamma': 1.8}
# Map = geemap.Map()
# Map.centerObject(piura_shp, 13)
# Map.addLayer(yearly_medians.first(), vis_basemap, 'last_image')
# Map


# JAXA forest non forest
https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF
https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_PALSAR_YEARLY_FNF


In [179]:
forest_2017_2020 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF4').filterDate('2017-01-01', '2020-12-31').map(lambda image: image.clip(piura_shp))
forest_2007_2016 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF').filterDate('2007-01-01', '2016-12-31').map(lambda image: image.clip(piura_shp))

video_args_4 = {
  'dimensions': 768,
  'region': piura_shp,
  'framesPerSecond': 2,
  'crs': 'EPSG:3857',
    'min': 1,
    'max': 4,
    'palette': ['00b200', '83ef62', 'ffff99', '0000ff']
}

video_args_3 = {
  'dimensions': 768,
  'region': piura_shp,
  'framesPerSecond': 2,
  'crs': 'EPSG:3857',
    'min': 1,
    'max': 3,
    'palette': ['00b200', '83ef62', 'ffff99']
}

# Get URL that will produce the animation when accessed.
gif_url = forest_2007_2016.getVideoThumbURL(video_args_3)
# gif_url

tst = forest_2017_2020.first()
tst = tst.visualize(
    bands = 'fnf',
    min = 1,
    max = 4,
    palette = ['00b200', '83ef62', 'ffff99', '0000ff']
)


Map.addLayer(tst, {}, 'forest_2017_2020')


# Export.image.toAsset({
#   image: SahelRainfallVis,
#   description: 'imageExport',
#   assetId: 'rf',
#   region: Sahel,
# });



# geemap.ee_export_image_collection(forest_2017_2020, out_dir=out_dir)
# geemap.ee_export_image_collection(forest_2007_2017, out_dir=out_dir)

# Define visualization parameters
forest_non_forest_vis = {
    'min': 1,
    'max': 3,
    'palette': ['00b200', '83ef62', 'ffff99']
}

# Set map center
Map.centerObject(piura_shp, 12)
# Map.addLayer(forest_2007_2016.select('JAXA/ALOS/PALSAR/YEARLY/FNF/2015'), forest_non_forest_vis, 'Forest/Non-Forest')
# Map

# Copernicus Land Cover
100m resolution
2015-01-01 to 2019-12-31

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#description

In [ ]:
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')
.select('discrete_classification')

# Landsat
Landsat provides 30m resolution from 1985 to today.

An annual composite is made to correct for cloud coverage, with instructions from the following video: https://www.youtube.com/watch?v=GzQ9HSXz9vE


In [ ]:

years = ee.List.sequence(1985, 2024)

# Map over the list of years and calculate geometric medians.
yearly_medians = ee.ImageCollection(years.map(calculate_median)).map(lambda image: image.clip(piura_shp))
yearly_medians_list = yearly_medians.toList(yearly_medians.size())

# Get the number of images.
num_images = yearly_medians_list.size().getInfo()

# Loop over the list and export each image.
for i in range(num_images):
    year = years.get(i).getInfo()
    task = ee.batch.Export.image.toDrive(
        image=ee.Image(yearly_medians_list.get(i)),
        description = 'Landsat_Satellite_30m_median_' + str(year),
        fileNamePrefix = 'Landsat_Satellite_30m_median_' + str(year),
        region=piura_shp,
        scale=3-,
        maxPixels=1e13
    )
    task.start()

# Dynamic World
10m land use, from 2015-06-27 to today. Near real time.


https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1#colab-python


In [ ]:
# Construct a collection of corresponding Dynamic World and Sentinel-2 for
# inspection. Filter the DW and S2 collections by region and date.
START = ee.Date('2021-04-02')
END = START.advance(1, 'day')

col_filter = ee.Filter.And(
    ee.Filter.bounds(ee.Geometry.Point(20.6729, 52.4305)),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))

# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)

# Display the Dynamic World visualization with the source Sentinel-2 image.
m = geemap.Map()
m.set_center(20.6729, 52.4305, 12)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

# ESA WorldCover
10m resolution
2020-01-01 - 2022-12-31

https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100
https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200

# ESRI Land Use Land Cover

https://livingatlas.arcgis.com/landcover/

In [ ]:
esri_lulc2020 = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m")


# ESRI Topography

All of these should have a finely traced but properly scaled grid with tick marks corresponding to decimal long./lat. degrees.

# NASA Elevation

In [128]:
# Import the dataset and select the elevation band.
dataset = ee.Image('NASA/NASADEM_HGT/001')
elevation = dataset.select('elevation')

# Add a white background image to the map.
background = ee.Image(1)
background_vis = {'min': 0, 'max': 1}
background_layer = background.visualize(**background_vis)

# Set elevation visualization properties.
elevation_vis = {'min': 0, 'max': 2000}

# Set elevation <= 0 as transparent.
elevation_masked = elevation.updateMask(elevation.gt(0)).clip(piura_shp)
elevation_masked
Map = geemap.Map()
Map.addLayer(background_layer, {}, 'Background')
Map.addLayer(elevation_masked, elevation_vis, 'Elevation')
Map.centerObject(piura_shp, 12)
Map

Map(center=[8.517382416611035, -81.00936813268046], controls=(WidgetControl(options=['position', 'transparent_…

# Hydrology

https://developers.google.com/earth-engine/datasets/catalog/MERIT_Hydro_v1_0_1

In [132]:
dataset = ee.Image('MERIT/Hydro/v1_0_1').clip(piura_shp)

visualization = {'bands': ['viswth']}

Map.centerObject(piura_shp, 12)

Map.addLayer(dataset, visualization, 'River width')
Map

Map(bottom=499687.0, center=[8.517382416611035, -81.00936813268046], controls=(WidgetControl(options=['positio…